<a href="https://colab.research.google.com/github/nitiksha/PySpark_code_practice/blob/main/union_subtract_intersect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Create a Spark session
spark = SparkSession.builder.appName("GoldMedalists").getOrCreate()

# Define the race results data
data = [
    (1, "100m", 2023, "Raj", "John", "Dsouza"),
    (2, "200m", 2023, "Paul", "Harry", "Raj"),
    (3, "300m", 2023, "John", "Barry", "Vinod"),
    (4, "500m", 2023, "Yuvi", "Saikat", "Narendar"),
    (5, "200m", 2023, "Chandra", "Santosh", "Raghav"),
    (6, "300m", 2023, "Yuvi", "Charles", "Thomas")
]

# Define the schema
columns = ["no", "race", "Year", "GOLD", "SILVER", "BRONZE"]

# Create a DataFrame
df = spark.createDataFrame(data, schema=columns)

none_df = df.filter((col("GOLD")==col("SILVER"))|(col("GOLD")==col("BRONZE"))|(col("SILVER")==col("BRONZE")))
none_df.show()

all_medalist = df.select("SILVER").union(df.select("BRONZE")).union(df.select("GOLD"))
all_medalist = all_medalist.groupBy("SILVER").count().withColumnRenamed("SILVER", "athlete").orderBy(col("count").desc())
all_medalist.show()

# Get set of all silver and bronze winners
other_medalists = df.select("SILVER").union(df.select("BRONZE")).distinct()

# Count gold medals per athlete
gold_medal_counts = df.groupBy("GOLD").count().withColumnRenamed("GOLD", "athlete").withColumnRenamed("count", "gold_medal_count")

# Filter athletes with only one gold medal and not present in other medals
single_gold_only = gold_medal_counts.filter(
    (~col("athlete").isin([row[0] for row in other_medalists.collect()]))
)

single_gold_only.show()

all_medalists = df.select("SILVER").union(df.select("BRONZE")).union(df.select("GOLD"))
all_medalists.show()


all_medalists1 = all_medalists.groupBy("SILVER").count().withColumnRenamed("SILVER", "athlete").withColumnRenamed("count", "more_medal_count")
all_medalists1.show()
all_medalists1.filter(col("count").cast("int")>1).show()





+---+----+----+----+------+------+
| no|race|Year|GOLD|SILVER|BRONZE|
+---+----+----+----+------+------+
+---+----+----+----+------+------+

+--------+-----+
| athlete|count|
+--------+-----+
|    John|    2|
|     Raj|    2|
|    Yuvi|    2|
|   Barry|    1|
|   Harry|    1|
| Charles|    1|
|  Saikat|    1|
| Santosh|    1|
|   Vinod|    1|
|  Dsouza|    1|
|Narendar|    1|
|  Raghav|    1|
|  Thomas|    1|
|    Paul|    1|
| Chandra|    1|
+--------+-----+

+-------+----------------+
|athlete|gold_medal_count|
+-------+----------------+
|   Paul|               1|
|Chandra|               1|
|   Yuvi|               2|
+-------+----------------+

+--------+
|  SILVER|
+--------+
|    John|
|   Harry|
|   Barry|
|  Saikat|
| Santosh|
| Charles|
|  Dsouza|
|     Raj|
|   Vinod|
|Narendar|
|  Raghav|
|  Thomas|
|     Raj|
|    Paul|
|    John|
|    Yuvi|
| Chandra|
|    Yuvi|
+--------+

+--------+----------------+
| athlete|more_medal_count|
+--------+----------------+
|   Barry|        